In [8]:
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
import math
import os
from datetime import date, timedelta, datetime
import time
from tqdm import tqdm
import pyodbc
import seaborn as sns
from scipy import stats
import xlsxwriter
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
start_time = time.perf_counter()

In [9]:
# price_data = d.fetch_price_data(no_of_years=20)
price_data = pd.read_csv('PriceData_23Jan.csv')
df =  price_data.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
df = price_data[price_data['Symbol'].isin(df['Symbol'])]
df[df['Date'] == '2025-01-22'].dropna()

,Date,Symbol,Open,High,Low,Close,Volume,Mcap,Liquidity,3y_Return
7044,2025-01-22,360ONE,1200.00,1211.50,1113.75,1148.95,408202.0,4.461106e+05,4.690037e+08,2.233405
11884,2025-01-22,3IINFOLTD,27.85,28.25,26.88,27.43,422842.0,4.650875e+03,1.159856e+07,-0.703299
17575,2025-01-22,3MINDIA,28960.00,29127.30,28560.15,29020.75,10265.0,3.269208e+05,2.978980e+08,0.163154
29616,2025-01-22,63MOONS,825.40,825.40,775.55,780.50,306802.0,3.596430e+04,2.394590e+08,2.417999
33091,2025-01-22,A2ZINFRA,22.10,23.30,21.12,21.35,321282.0,3.760159e+03,6.859371e+06,1.468208
...,...,...,...,...,...,...,...,...,...,...
7325402,2025-01-22,ZENTEC,2145.00,2148.95,1930.90,1974.20,780773.0,1.782512e+05,1.541402e+09,8.165274
7333593,2025-01-22,ZODIACLOTH,122.00,125.05,118.51,121.73,12073.0,3.164215e+03,1.469646e+06,0.137664
7338691,2025-01-22,ZOMATO,207.15,218.20,203.85,216.45,178256196.0,2.088818e+06,3.858355e+10,0.578775
7343668,2025-01-22,ZUARI,210.90,210.90,202.50,207.66,300084.0,8.733766e+03,6.231544e+07,0.773356


In [10]:
# Function to calculate log returns
def calculate_log_returns(df):
    df['LogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
    return df.dropna()
# Apply log return calculation
df = df.groupby('Symbol', group_keys=False).apply(calculate_log_returns)

# Function to calculate annualized standard deviation
def calculate_annualized_std(group, window=252):
    return group['LogReturn'].rolling(window).std() * np.sqrt(window)

period = 252
# Function to calculate annualized downside volatility
def calculate_annualized_downside_vol(series, period, annual_trading_days=252):
    # Isolate negative returns (only downside)
    negative_returns = series.clip(upper=0)
    # Calculate rolling downside standard deviation and annualize it
    downside_std = negative_returns.rolling(period).std() * np.sqrt(period)
    return downside_std

# Apply annualized standard deviation calculation within each group
df['AnnualizedStd'] = df.groupby('Symbol', group_keys=False).apply(
    lambda group: calculate_annualized_std(group)
)
# # Apply log return calculation
# df = df.groupby('Symbol', group_keys=False).apply(calculate_log_returns)
# # Calculate annualized downside volatility
# df['AnnualizedDownVol'] = df.groupby('Symbol', group_keys=False).apply(
#     lambda group: calculate_annualized_downside_vol(group['LogReturn'], period=252)
# )
# Reset index if needed
df = df.reset_index()


# Calculate the mean and standard deviation for AnnualizedStd across the universe on each date
df['mu_AnnualizedStd'] = df.groupby('Date')['AnnualizedStd'].transform('mean')
df['sigma_AnnualizedStd'] = df.groupby('Date')['AnnualizedStd'].transform('std')

# Calculate z-score for volatility
df['Z_AnnualizedStd'] = (df['AnnualizedStd'] - df['mu_AnnualizedStd']) / df['sigma_AnnualizedStd']

# Invert the z-score for "Low Vol Score" (higher score for lower volatility)
df['LowVolScore'] = -df['Z_AnnualizedStd']

# Normalize the Low Vol Score
# Positive z-scores: 1 + score, Negative z-scores: (1 - score) ** -1
df['NormalizedLowVolScore'] = np.where(
    df['LowVolScore'] >= 0,
    1 + df['LowVolScore'],
    (1 - df['LowVolScore']) ** -1
)
df = df.reset_index()
df = df.sort_values(by=['Date','Mcap'], ascending=[True,False])
df = df[['Date','Symbol','Mcap','Close','NormalizedLowVolScore']]
df = df.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))

In [ ]:
df = df.sort_values(by=['Date','NormalizedLowVolScore'], ascending=[True,False])

,Date,Symbol,Mcap,Close,NormalizedLowVolScore
2993750,2025-01-22,PGHH,4.769602e+05,14693.45,3.122935
2735043,2025-01-22,NESTLEIND,2.128666e+06,2207.80,3.008872
3761223,2025-01-22,SUNPHARMA,4.316284e+06,1798.95,2.980071
1013276,2025-01-22,DRREDDY,1.081622e+06,1296.25,2.881255
1743453,2025-01-22,ICICIBANK,8.481188e+06,1200.45,2.834490
...,...,...,...,...,...
1469490,2025-01-22,GRSE,1.767136e+05,1542.65,0.373175
3226138,2025-01-22,RAYMOND,1.013452e+05,1522.30,0.365934
3269879,2025-01-22,RELINFRA,1.086984e+05,274.40,0.343119
1964873,2025-01-22,ITI,3.390490e+05,352.85,0.334300


In [28]:
# theme_df = d.fetch_data_from_database(table_name='SectorThemeGics')[['Symbol', 'Theme']]
# theme = df[['Date', 'Symbol', 'AnnualizedStd','AnnualizedDownVol']].merge(theme_df, on='Symbol', how='inner')

# # Group by Date and Theme, and calculate the mean of each momentum score and the count of symbols
# df_grouped = theme.groupby(['Date', 'Theme']).agg(
#     AnnualizedStdTheme=('AnnualizedStd', 'mean'),
#     DownVolTheme=('AnnualizedDownVol', 'mean')).reset_index()
# # Define the columns to rank
# theme_mom_columns = ['AnnualizedStdTheme','DownVolTheme']

# # Calculate the percentile rank for each MoM column, for each date
# for col in theme_mom_columns:
#     df_grouped[f'{col}_Rank'] = df_grouped.groupby('Date')[col].rank(pct=True, ascending=False)
# df_grouped = df_grouped[['Date','Theme','AnnualizedStdTheme_Rank','DownVolTheme_Rank']]

# final_mapped_df = theme.merge(df_grouped, on=['Date', 'Theme'], how='left')
# final_mapped_df = final_mapped_df.sort_values(by='Date')
# final_mapped_df.drop(columns='Theme', inplace=True)
# final_mapped_df

In [13]:
# Define the columns to rank
vol_columns = ['NormalizedLowVolScore']
# Calculate the percentile rank for each MoM column, for each date
for col in vol_columns:
    df[f'{col}_Rank'] = df.groupby('Date')[col].rank(pct=True, ascending=True)
# Define the columns to drop
# Drop the specified MoM columns from the DataFrame
# df = df.drop(columns=vol_columns)
# Define the columns to check for NaN values
rank_columns = ['AnnualizedStd_Rank']
# rank_columns = ['AnnualizedStdTheme_Rank','DownVolTheme_Rank','AnnualizedStd_Rank','AnnualizedDownVol_Rank']
# Drop rows where all specified rank columns are NaN
# df = df.dropna(subset=rank_columns, how='all')
df

,Date,Symbol,Mcap,Close,NormalizedLowVolScore,NormalizedLowVolScore_Rank
3255305,1998-01-05,RELIANCE,161025.084547,17.360683,NaN,NaN
3884768,1998-01-05,TATASTEEL,52220.172171,7.964170,NaN,NaN
3255306,1998-01-06,RELIANCE,159717.835383,17.219744,NaN,NaN
3884769,1998-01-06,TATASTEEL,51410.271721,7.840651,NaN,NaN
3255307,1998-01-07,RELIANCE,161305.209368,17.390884,NaN,NaN
...,...,...,...,...,...,...
1946302,2025-01-22,ISGEC,86459.674334,1175.850000,NaN,NaN
174589,2025-01-22,AMIORG,80720.710369,1971.950000,NaN,NaN
3415785,2025-01-22,SANSERA,77962.681321,1259.050000,NaN,NaN
2296480,2025-01-22,KPIGREEN,76698.735178,389.550000,NaN,NaN


In [ ]:
Y = final_mapped_df.filter(["Symbol", "Date", "AnnualizedStdTheme_Rank","StdTheme_Rank","AnnualizedStd_Rank","Std_Rank"])
Y = Y.pivot_table(index='Date',columns='Symbol',values='AnnualizedStd_Rank').reset_index()
## Convert the long from Dataframe to wide form dataframe
## Shifting the Date column
Y["Date"] = Y["Date"].shift(-1)
## Fill NaN value of date with todays date.
Y.iloc[-1, Y.columns.get_loc("Date")] = pd.to_datetime(date.today())
## Re-converting the wide form dataframe to long form dataframe
Y = Y.melt(id_vars='Date', value_name = "AnnualizedStd_Rank")
## Drop na and reset the index
Y = Y.dropna().reset_index(drop = True)
Y = Y[Y["Date"] >= "2006-01-01"].reset_index(drop = True).copy()
Y

In [8]:
# # Define the columns to rank
# vol_columns = ['AnnualizedStd','AnnualizedDownVol']
# # Calculate the percentile rank for each MoM column, for each date
# for col in vol_columns:
#     final_mapped_df[f'{col}_Rank'] = final_mapped_df.groupby('Date')[col].rank(pct=True, ascending=False)
# # Define the columns to drop
# # Drop the specified MoM columns from the DataFrame
# final_mapped_df = final_mapped_df.drop(columns=vol_columns)
# # Define the columns to check for NaN values
# rank_columns = ['AnnualizedStdTheme_Rank','DownVolTheme_Rank','AnnualizedStd_Rank','AnnualizedDownVol_Rank']
# # Drop rows where all specified rank columns are NaN
# final_mapped_df = final_mapped_df.dropna(subset=rank_columns, how='all')
# final_mapped_df

In [9]:
# final_mapped_df = final_mapped_df.sort_values(by=['Date','AnnualizedStd_Rank'], ascending=[True,False])
# final_mapped_df['Date'] = final_mapped_df.groupby('Symbol')['Date'].shift(-1)
# final_mapped_df = final_mapped_df.dropna(subset=['Date'])
# # final_mapped_df['Date'] = final_mapped_df['Date'].fillna(pd.to_datetime('2024-10-24'))
# final_mapped_df

In [10]:
# final_mapped_df['AverageThemeVol_Rank'] = (final_mapped_df['AnnualizedStdTheme_Rank'] + final_mapped_df['DownVolTheme_Rank'])/2
# final_mapped_df['AverageVol_Rank'] = (final_mapped_df['AnnualizedStd_Rank'] + final_mapped_df['AnnualizedDownVol_Rank'])/2
# final_mapped_df

In [ ]:
# final_mapped_df[final_mapped_df['Date'] == '2024-08-22'].sort_values(by='AnnualizedDownVol_Rank', ascending=False).head(30).to_csv('LastVol.csv')

In [21]:
# a = final_mapped_df[final_mapped_df['Date'] == '2024-10-18'].sort_values(by='AnnualizedDownVol_Rank',ascending=False).head(30)
# a.to_csv('Actual.csv')

In [15]:
# final_mapped_df.to_csv('FullVol.csv')

In [22]:
# Y = final_mapped_df.filter(["Symbol", "Date", "AnnualizedStdTheme_Rank","StdTheme_Rank","AnnualizedStd_Rank","Std_Rank"])
# Y = Y.pivot_table(index='Date',columns='Symbol',values='AnnualizedStd_Rank').reset_index()
# ## Convert the long from Dataframe to wide form dataframe
# ## Shifting the Date column
# Y["Date"] = Y["Date"].shift(-1)
# ## Fill NaN value of date with todays date.
# Y.iloc[-1, Y.columns.get_loc("Date")] = pd.to_datetime(date.today())
# ## Re-converting the wide form dataframe to long form dataframe
# Y = Y.melt(id_vars='Date', value_name = "AnnualizedStd_Rank")
# ## Drop na and reset the index
# Y = Y.dropna().reset_index(drop = True)
# Y = Y[Y["Date"] >= "2006-01-01"].reset_index(drop = True).copy()
# Y

In [ ]:
# df['Date'] = df.groupby('Symbol')['Date'].shift(-1)
# df = df.dropna(subset=['Date'])
# df = df.dropna()
# df = df[['Date','Symbol','AnnualizedStd']]
# df.to_csv('Volsleeve.csv')

In [11]:
price_data['Date'] = pd.to_datetime(price_data['Date'], errors='coerce')
qualf = df.dropna()
qualf = qualf.sort_values(by=['Date','NormalizedLowVolScore'], ascending=[True,False])
top_25_stocks_each_day = qualf.groupby('Date').tail(25)
top_25_stocks_each_day['Date'] = pd.to_datetime(top_25_stocks_each_day['Date'])
rebalanceFrequency=2
from rebalancedate import *
op=rebalancedates(price_data,'2006-06-02',17, rebalanceFrequency)
rebalanceDates=op[1]
df_rebalance_dates = pd.DataFrame({'Date': pd.to_datetime(rebalanceDates)})
df_rebalance_dates
top_25_stocks_each_day = top_25_stocks_each_day[top_25_stocks_each_day['Date'].isin(df_rebalance_dates['Date'])].reset_index(drop=True)
top_25_stocks_each_day.to_csv('LowVolShort.csv')

In [ ]:
# top_stocks = 30
# df_stocks = df[['Date', 'Symbol', 'AnnualizedStd_Rank']].copy()
# df_stocks = df_stocks.groupby('Date').apply(lambda x: x.nsmallest(top_stocks, 'AnnualizedStd_Rank')).reset_index(drop=True)
# df_stocks = df_stocks[['Date', 'Symbol', 'AnnualizedStd_Rank']].dropna(subset=['AnnualizedStd_Rank'])
# df_stocks['InverseVol'] = 1/df_stocks['AnnualizedStd_Rank']
# #Calculate sum of InverseVol for each date
# df_stocks['SumInverseVol'] = df_stocks.groupby('Date')['InverseVol'].transform('sum')
# # Calculate InverseVol / Sum(InverseVol) for each row
# df_stocks['VolWeight'] = df_stocks['InverseVol'] / df_stocks['SumInverseVol']
# df_stocks = df_stocks[['Date','Symbol','VolWeight']]
# df_stocks['VolWeight'] = df_stocks['VolWeight']*100
# df_stocks

In [20]:
x = df[df['Date'] =='2025-01-22'].dropna()
x[x['Symbol'] == 'TECHM']

,Date,Symbol,Mcap,Close,NormalizedLowVolScore,NormalizedLowVolScore_Rank
3923732,2025-01-22,TECHM,1.648310e+06,1683.95,2.425097,0.882609


In [ ]:
# df_stocks[df_stocks['Date'] == '2024-10-17'].sort_values(by='VolWeight', ascending=False).to_csv('InverseWeight30.csv')

In [21]:
df['mean_vol'] = df.groupby('Date')['AnnualizedStd'].transform('mean')
df['STD_vol'] = df.groupby('Date')['AnnualizedStd'].transform('std')
# Convert Alpha to Z-score
# df['Z_Vol'] = (df['AnnualizedStd'] - df['mean_vol']) / df['STD_vol']
# # Calculate Normalized Alpha Quality Score
# df['Vol_Score'] = np.where(df['Z_Vol'] >= 0, 
#                                      1 + df['Z_Vol'], 
#                                      (1 - df['Z_Vol'])**-1)
# df

# Calculate reversed Z-score for Volatility
df['Z_Vol'] = (df['mean_vol'] - df['AnnualizedStd']) / df['STD_vol']

# Calculate Normalized Volatility Quality Score (higher std => lower score)
df['Vol_Score'] = np.where(df['Z_Vol'] >= 0, 
                           1 + df['Z_Vol'], 
                           (1 - df['Z_Vol'])**-1)

In [10]:
rebalanceFrequency=2
from rebalance import *
op=rebalanceDates(price_data,'2006-06-02',17, rebalanceFrequency)
rebalanceDates=op
df_rebalance_dates = pd.DataFrame({'Timestamp': pd.to_datetime(rebalanceDates)})
df_rebalance_dates.rename(columns={'Timestamp': 'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])

df = df[df['Date'].isin(df_rebalance_dates['Date'])].reset_index(drop=True)
top_stocks = 10
df_stocks = df.copy()
df_stocks = df_stocks.groupby('Date').apply(lambda x: x.nlargest(top_stocks, 'Vol_Score')).reset_index(drop=True)
df_stocks

,Date,Symbol,Mcap,Close,AnnualizedStd,mean_vol,STD_vol,Z_Vol,Vol_Score
0,2006-06-19,MORGANSTAN,2.016612e+04,33.600000,0.252497,0.481456,0.124060,1.845542,2.845542
1,2006-06-19,BIOCON,3.438000e+04,28.650000,0.279032,0.481456,0.124060,1.631663,2.631663
2,2006-06-19,KANSAINER,1.816401e+04,23.736667,0.279215,0.481456,0.124060,1.630181,2.630181
3,2006-06-19,DREDGECORP,1.468600e+04,524.500000,0.281186,0.481456,0.124060,1.614296,2.614296
4,2006-06-19,GILLETTE,2.348417e+04,720.700000,0.294859,0.481456,0.124060,1.504080,2.504080
...,...,...,...,...,...,...,...,...,...
1105,2024-10-17,ITC,6.114965e+06,488.900000,0.190220,0.400311,0.244778,0.858290,1.858290
1106,2024-10-17,DRREDDY,1.118751e+06,1340.810000,0.195868,0.400311,0.244778,0.835218,1.835218
1107,2024-10-17,BATAINDIA,1.844370e+05,1435.000000,0.196166,0.400311,0.244778,0.833999,1.833999
1108,2024-10-17,NESTLEIND,2.293441e+06,2378.700000,0.200413,0.400311,0.244778,0.816650,1.816650


In [30]:
df = df[['Date','Symbol','NormalizedLowVolScore','NormalizedLowVolScore_Rank']]
df.to_csv('Lowvol.csv', index=False)

In [14]:
churnDf = df_stocks.groupby("Date")["Symbol"].apply(set).rename("Current").reset_index()
churnDf["Previous"] = churnDf["Current"].shift()
churnDf = churnDf.dropna()
churnDf["Churn"] = churnDf.apply(lambda x : len(x["Current"] - x["Previous"])/len(x["Previous"]), axis = 1)
churnDf["Churn"].mean()

0.25181818181818183